### 피봇테이블

`pivot`

피봇테이블(pivot table)이란 데이터 열 중에서 두 개의 열을 각각 행 인덱스, 열 인덱스로 사용하여 데이터를 조회하여 펼쳐놓은 것을 말한다.

판다스는 피봇테이블을 만들기 위한 pivot 메서드를 제공한다.
첫번째 인수로는 행 인덱스로 사용할 열 이름, 두번째 인수로는 열 인덱스로 사용할 열 이름, 그리고 마지막으로 데이터로 사용할 열 이름을 넣는다.

판다스는 지정된 두 열을 각각 행 인덱스와 열 인덱스로 바꾼 후
행 인덱스의 라벨 값이 첫번째 키의 값과 같고 열 인덱스의 라벨 값이 두번째 키의 값과 같은 데이터를 찾아서 해당 칸에 넣는다.
만약 주어진 데이터가 존재하지 않으면 해당 칸에 NaN 값을 넣는다.

In [3]:
import pandas as pd

In [4]:
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [5]:
df2 = df1[['도시', '연도', '인구']]
df2

,도시,연도,인구
0,서울,2015,9904312
1,서울,2010,9631482
2,서울,2005,9762546
3,부산,2015,3448737
4,부산,2010,3393191
5,부산,2005,3512547
6,인천,2015,2890451
7,인천,2010,263203


In [6]:
df1.pivot?

In [7]:
# 피봇 - 도시=행인덱스, 연도=열인덱스, 인구=값
df2.pivot(index='도시', columns='연도', values='인구')

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [8]:
df2.pivot(index='연도', columns='도시', values='인구') #행열 바꾸기

도시,부산,서울,인천
연도,,,
2005,3512547.0,9762546.0,NaN
2010,3393191.0,9631482.0,263203.0
2015,3448737.0,9904312.0,2890451.0


이 피봇테이블의 값 3512547은 "도시"가 부산이고 "연도"가 2005년인 데이터를 "인구"열에서 찾은 값이다.
2005년 인천의 인구는 데이터에 없기 때문에 NaN으로 표시된다.

피봇테이블은 다음과 같이 set_index 명령(행인덱스 생성)과 unstack 명령(행>열 교환)을 사용해서 만들 수도 있다.

In [9]:
df1.set_index(['도시', '연도'])

인구   지역
도시 연도                
서울 2015  9904312  수도권
   2010  9631482  수도권
   2005  9762546  수도권
부산 2015  3448737  경상권
   2010  3393191  경상권
   2005  3512547  경상권
인천 2015  2890451  수도권
   2010   263203  수도권

In [10]:
df1.set_index(['도시', '연도'])[['인구']] #칼럼이 하나면 시리즈 출력, 차원으로 올리면 데이터프레임 출력

인구
도시 연도           
서울 2015  9904312
   2010  9631482
   2005  9762546
부산 2015  3448737
   2010  3393191
   2005  3512547
인천 2015  2890451
   2010   263203

In [11]:
df1.set_index(['도시', '연도'])[['인구']].unstack()

인구                      
연도       2005       2010       2015
도시                                 
부산  3512547.0  3393191.0  3448737.0
서울  9762546.0  9631482.0  9904312.0
인천        NaN   263203.0  2890451.0

행 인덱스나 열 인덱스를 리스트로 주는 경우에는 다중 인덱스 피봇 테이블을 생성한다. (주의: 판다스 버전 1.1 미만에서는 버그로 인해 동작하지 않는다.)

In [12]:
df1.pivot(index= ["지역", "도시"], columns="연도", values="인구")

연도           2005       2010       2015
지역  도시                                 
경상권 부산  3512547.0  3393191.0  3448737.0
수도권 서울  9762546.0  9631482.0  9904312.0
    인천        NaN   263203.0  2890451.0

행 인덱스와 열 인덱스는 데이터를 찾는 키(key)의 역할을 한다. 따라서 키 값으로 데이터가 단 하나만 찾아져야 한다.

만약 행 인덱스와 열 인덱스 조건을 만족하는 데이터가 2개 이상인 경우에는 에러가 발생한다.
예를 들어 위 데이터프레임에서 ("지역", "연도")를 키로 하면 ("수도권", "2015")에 해당하는 값이 두 개 이상이므로 다음과 같이 에러가 발생한다.

In [13]:
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [14]:
try:
    df1.pivot(index ='지역', columns = '연도', values = '인구')
except ValueError as e:
    print('ValueError:', e)

ValueError: Index contains duplicate entries, cannot reshape


### 그룹분석
만약 키가 지정하는 조건에 맞는 데이터가 하나 이상이라서 데이터 그룹을 이루는 경우에는 그룹의 특성을 보여주는 그룹분석(group analysis)을 해야 한다.

그룹분석은 피봇테이블과 달리 키에 의해서 결정되는 데이터가 여러개가 있을 경우 미리 지정한 연산을 통해 그 그룹 데이터의 대표값을 계산한다.
판다스에서는 groupby 메서드를 사용하여 다음처럼 그룹분석을 한다.

1. 분석하고자 하는 시리즈나 데이터프레임에 groupby 메서드를 호출하여 그룹화를 한다.
2. 그룹 객체에 대해 그룹연산을 수행한다.

#### groupby 메서드

groupby 메서드는 데이터를 그룹 별로 분류하는 역할을 한다. groupby 메서드의 인수로는 다음과 같은 값을 사용한다.

* 열 또는 열의 리스트
* 행 인덱스

연산 결과로 그룹 데이터를 나타내는 GroupBy 클래스 객체를 반환한다. 이 객체에는 그룹별로 연산을 할 수 있는 그룹연산 메서드가 있다.

그룹연산 메서드
groupby 결과, 즉 GroupBy 클래스 객체의 뒤에 붙일 수 있는 그룹연산 메서드는 다양하다. 다음은 자주 사용되는 그룹연산 메서드들이다.

* size, count: 그룹 데이터의 갯수
* mean, median, min, max: 그룹 데이터의 평균, 중앙값, 최소, 최대
* sum, prod, std, var, quantile : 그룹 데이터의 합계, 곱, 표준편차, 분산, 사분위수
* first, last: 그룹 데이터 중 가장 첫번째 데이터와 가장 나중 데이터

이 외에도 많이 사용되는 것으로는 다음과 같은 그룹연산이 있다.

* agg, aggregate

    * 만약 원하는 그룹연산이 없는 경우 함수를 만들고 이 함수를 agg에 전달한다.
    * 또는 여러가지 그룹연산을 동시에 하고 싶은 경우 함수 이름 문자열의 리스트를 전달한다.
* describe
    * 하나의 그룹 대표값이 아니라 여러개의 값을 데이터프레임으로 구한다.
* apply
    * describe 처럼 하나의 대표값이 아닌 데이터프레임을 출력하지만 원하는 그룹연산이 없는 경우에 사용한다.
* transform
    * 그룹에 대한 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터 자체를 변형한다.

예를 들어 다음과 같은 데이터가 있을 때 key1의 값(A 또는 B)에 따른 data1의 평균은 어떻게 구할까?

In [15]:
import numpy as np
np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [16]:
df2.groupby?

In [17]:
groups = df2.groupby(df2.key1)
groups

이 GroupBy 클래스 객체에는 각 그룹 데이터의 인덱스를 저장한 groups 속성이 있다.

In [18]:
groups.groups

{'A': [0, 1, 4], 'B': [2, 3]}

A그룹과 B그룹 데이터의 합계를 구하기 위해 sum이라는 그룹연산을 한다.

In [19]:
groups.sum()

,key2,data1,data2
key1,,,
A,onetwoone,8,80
B,onetwo,7,70


GroupBy 클래스 객체를 명시적으로 얻을 필요가 없다면 groupby 메서드와 그룹연산 메서드를 연속으로 호출한다.
다음 예제는 열 data1에 대해서만 그룹연산을 하는 코드이다.

In [20]:
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [21]:
df2.data1.groupby(df2.key1).sum() #필요한 연산만 하는 좋은 방식

key1
A    8
B    7
Name: data1, dtype: int64

데이터를 그룹으로 나눈 GroupBy 클래스 객체 또는 그룹분석한 결과에서 data1만 뽑아도 된다.

In [22]:
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [23]:
df2.groupby(df2.key1)["data1"].sum()   # `GroupBy` 클래스 객체에서 data1만 선택하여 분석하는 경우

key1
A    8
B    7
Name: data1, dtype: int64

In [24]:
df2.groupby(df2.key1).sum()["data1"]  # 전체 데이터를 분석한 후 data1만 선택한 경우

key1
A    8
B    7
Name: data1, dtype: int64

연습 문제

key1의 값을 기준으로 data1의 값을 분류하여 합계를 구한 결과를 시리즈가 아닌 데이터프레임으로 구한다.

In [25]:
df2.groupby(df2.key1)[["data1"]].sum()

,data1
key1,
A,8
B,7


In [26]:
# 복합 키 (key1, key2) 값에 따른 data1의 합계를 구하자.
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [27]:
df2.data1.groupby([df2.key1, df2.key2]).sum()

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

In [28]:
# unstack 명령으로 피봇 데이블 형태로 만들수도 있다.
df2.data1.groupby([df2.key1, df2.key2]).sum().unstack('key2')

key2,one,two
key1,,
A,6,2
B,3,4


In [29]:
# 그룹분석 기능을 사용하면 위의 인구 데이터로부터 지역별 합계를 구할 수도 있다.
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [30]:
df1['인구'].groupby(df1['지역']).sum()

지역
경상권    10354475
수도권    32451994
Name: 인구, dtype: int64

In [31]:
df1["인구"].groupby([df1["지역"], df1["연도"]]).sum()

지역   연도  
경상권  2005     3512547
     2010     3393191
     2015     3448737
수도권  2005     9762546
     2010     9894685
     2015    12794763
Name: 인구, dtype: int64

In [32]:
df1["인구"].groupby([df1["지역"], df1["연도"]]).sum().unstack("연도")

연도,2005,2010,2015
지역,,,
경상권,3512547,3393191,3448737
수도권,9762546,9894685,12794763


agg

다음 데이터는 150 송이의 붓꽃(iris)에 대해 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width), 꽃잎폭(sepal_width), 꽃잎폭(sepal_width)을 측정한 데이터이다.

In [33]:
import seaborn as sns
iris = sns.load_dataset("iris")
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [34]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


각 붓꽃 종별로 가장 큰 값과 가장 작은 값의 비율을 구해보자.
이러한 계산을 하는 그룹연산 메서드는 없으므로 직접 만든 후 agg 메서드를 적용한다.

In [35]:
def peak_to_peak_ratio(x):
    return x.max() / x.min()

iris.groupby(iris.species).agg(peak_to_peak_ratio)

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


`describe`

describe 메서드를 사용하면 다양한 기술 통계(descriptive statistics)값을 한 번에 구한다.
그룹별로 하나의 스칼라 값이 아니라 하나의 데이터프레임이 생성된다는 점에 주의하라.

In [36]:
iris.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [37]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [38]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [39]:
iris.groupby(iris.species).describe() #그룹바이에도 디스크라이브를 넣을 수 있음

sepal_length                                              \
                  count   mean       std  min    25%  50%  75%  max   
species                                                               
setosa             50.0  5.006  0.352490  4.3  4.800  5.0  5.2  5.8   
versicolor         50.0  5.936  0.516171  4.9  5.600  5.9  6.3  7.0   
virginica          50.0  6.588  0.635880  4.9  6.225  6.5  6.9  7.9   

           sepal_width         ... petal_length      petal_width         \
                 count   mean  ...          75%  max       count   mean   
species                        ...                                        
setosa            50.0  3.428  ...        1.575  1.9        50.0  0.246   
versicolor        50.0  2.770  ...        4.600  5.1        50.0  1.326   
virginica         50.0  2.974  ...        5.875  6.9        50.0  2.026   

                                               
                 std  min  25%  50%  75%  max  
species                                        
setosa      0.105386  0.1  0.2  0.2  0.3  0.6  
versicolor  0.197753  1.0  1.2  1.3  1.5  1.8  
virginica   0.274650  1.4  1.8  2.0  2.3  2.5  

[3 rows x 32 columns]

In [40]:
iris.groupby(iris.species).describe().T #행렬전환 (트랜스포즈)

species                setosa  versicolor  virginica
sepal_length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
sepal_width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
petal_length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
petal_width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000

`apply`

apply 메서드를 사용하면 describe 메서드처럼 하나의 그룹에 대해 하나의 대표값(스칼라 값)을 구하는 게 아니라 데이터프레임을 만들 수 있다.

예를 들어 다음처럼 각 붓꽃 종별로 가장 꽃잎 길이(petal length)가 큰 3개의 데이터를 뽑아낼 수도 있다.

In [41]:
iris.sort_values(by="petal_length", ascending=False)[:3]

,sepal_length,sepal_width,petal_length,petal_width,species
118,7.7,2.6,6.9,2.3,virginica
122,7.7,2.8,6.7,2.0,virginica
117,7.7,3.8,6.7,2.2,virginica


In [42]:
def top3_petal_length(df):
    return df.sort_values(by="petal_length", ascending=False)[:3]

iris.groupby(iris.species).apply(top3_petal_length)

sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     24            4.8          3.4           1.9          0.2   
           44            5.1          3.8           1.9          0.4   
           23            5.1          3.3           1.7          0.5   
versicolor 83            6.0          2.7           5.1          1.6   
           77            6.7          3.0           5.0          1.7   
           72            6.3          2.5           4.9          1.5   
virginica  118           7.7          2.6           6.9          2.3   
           117           7.7          3.8           6.7          2.2   
           122           7.7          2.8           6.7          2.0   

                   species  
species                     
setosa     24       setosa  
           44       setosa  
           23       setosa  
versicolor 83   versicolor  
           77   versicolor  
           72   versicolor  
virginica  118   virginica  
           117   virginica  
           122   virginica

`transform`

transform 메서드는 그룹별 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터프레임 자체를 변화시킨다.

따라서 만들어진 데이터프레임의 크기는 원래 데이터프레임과 같다.
예를 들어 다음처럼 각 붓꽃 꽃잎길이가 해당 종 내에서 대/중/소 어느 것에 해당되는지에 대한 데이터프레임을 만들 수도 있다.

In [43]:
def q3cut(s):
    return pd.qcut(s, 3, labels=["소", "중", "대"]).astype(str)


iris["petal_length_class"] = iris.groupby(iris.species).petal_length.transform(q3cut)
iris[["petal_length", "petal_length_class"]].tail(10)

,petal_length,petal_length_class
140,5.6,중
141,5.1,소
142,5.1,소
143,5.9,대
144,5.7,중
145,5.2,소
146,5.0,소
147,5.2,소
148,5.4,중
149,5.1,소


연습 문제

붓꽃(iris) 데이터에서 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width) 등의 평균을 구하라.
만약 붓꽃 종(species)이 표시되지 않았을 때 이 수치들을 이용하여 붓꽃 종을 찾아낼 수 있을지 생각하라.

In [44]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [45]:
iris.groupby('species').mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


### pivot_table

*  pivot 명령과 groupby 명령의 중간 성격을 가지는 pivot_table 명령

 groupby 명령처럼 그룹분석을 하지만 최종적으로는 pivot 명령처럼 피봇테이블을 만든다. groupby 명령의 결과에 unstack을 자동 적용하여 2차원적인 형태로 변형한다.
 
* pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, margins_name='All')

예를 들어 위에서 만들었던 피봇테이블은 pivot_table 명령으로 다음과 같이 만들 수도 있다. 인수의 순서에 주의하라.

In [46]:
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [47]:
df1.pivot_table(values="인구", index="도시", columns="연도") # 인자를 안써도 되지만, 헷갈리니 꼭 기입해주자

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


margins=True 인수를 주면 aggfunc로 주어진 분석 방법을 해당 열의 모든 데이터, 해당 행의 모든 데이터 그리고 전체 데이터에 대해 적용한 결과를 같이 보여준다.

aggfunc가 주어지지 않았으면 평균을 계산한다.

In [48]:
df1.pivot_table(values="인구", index="도시", columns="연도", margins=True, margins_name="평균") # aggfunc 디폴트 평균계산

연도,2005,2010,2015,평균
도시,,,,
부산,3512547.0,3393191.0,3448737.0,3.451492e+06
서울,9762546.0,9631482.0,9904312.0,9.766113e+06
인천,NaN,263203.0,2890451.0,1.576827e+06
평균,6637546.5,4429292.0,5414500.0,5.350809e+06


이 결과에서 가장 오른쪽 합계 열의 첫번째 값 3451492은 모든 부산 인구 데이터의 평균,
두번째 값 9766113은 모든 서울 인구 데이터의 평균이다.
가장 아래의 합계 행의 첫번째 값은 2005년 데이터의 평균값,
두번째 값은 2010년 데이터의 평균값이다.
가장 오른쪽 아래의 값 5350809는 전체 데이터의 평균값이다. 다음 계산을 통해 이를 확인할 수 있다.

In [49]:
df1['인구'].mean()

5350808.625

행 인덱스나 열 인덱스에 리스트를 넣으면 다중 인덱스 테이블을 만든다.

In [50]:
df1.pivot_table("인구", index=["연도", "도시"])

인구
연도   도시         
2005 부산  3512547
     서울  9762546
2010 부산  3393191
     서울  9631482
     인천   263203
2015 부산  3448737
     서울  9904312
     인천  2890451

식당에서 식사 후 내는 팁(tip)과 관련된 데이터를 이용하여 좀더 구체적으로 그룹분석 방법을 살펴본다.
우선 Seaborn 패키지에 설치된 샘플 데이터를 로드한다. 이 데이터프레임에서 각각의 컬럼은 다음을 뜻한다.

total_bill: 식사대금
tip: 팁
sex: 성별
smoker: 흡연/금연 여부
day: 요일
time: 시간
size: 인원

In [51]:
tips = sns.load_dataset("tips")
tips.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


분석의 목표는 식사 대금 대비 팁의 비율이 어떤 경우에 가장 높아지지는 찾는 것이다.
우선 식사대금와 팁의 비율을 나타내는 tip_pct를 추가하자.

In [52]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.tail()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204
243,18.78,3.00,Female,No,Thur,Dinner,2,0.159744


다음으로 각 열의 데이터에 대해 간단히 분포를 알아본다.



In [53]:
tips.describe()

,total_bill,tip,size,tip_pct
count,244.000000,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672,0.160803
std,8.902412,1.383638,0.951100,0.061072
min,3.070000,1.000000,1.000000,0.035638
25%,13.347500,2.000000,2.000000,0.129127
50%,17.795000,2.900000,2.000000,0.154770
75%,24.127500,3.562500,3.000000,0.191475
max,50.810000,10.000000,6.000000,0.710345


우선 성별로 나누어 데이터 갯수를 세어본다.



In [54]:
tips.groupby("sex").count()

,total_bill,tip,smoker,day,time,size,tip_pct
sex,,,,,,,
Male,157,157,157,157,157,157,157
Female,87,87,87,87,87,87,87


데이터 갯수의 경우 NaN 데이터가 없다면 모두 같은 값이 나올 것이다.
이 때는 size 명령을 사용하면 더 간단히 표시된다. size 명령은 NaN이 있어도 상관하지 않는다.

In [55]:
tips.groupby("sex").size()

sex
Male      157
Female     87
dtype: int64

이번에는 성별과 흡연유무로 나누어 데이터의 갯수를 알아본다.


In [56]:
tips.groupby(["sex", "smoker"]).count()

total_bill  tip  day  time  size  tip_pct
sex    smoker                                           
Male   Yes             60   60   60    60    60       60
       No              97   97   97    97    97       97
Female Yes             33   33   33    33    33       33
       No              54   54   54    54    54       54

In [57]:
tips.groupby(["sex", "smoker"]).count()['total_bill']

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
Name: total_bill, dtype: int64

In [58]:
tips.groupby(["sex", "smoker"]).size()

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

좀 더 보기 좋도록 피봇 데이블 형태로 바꿀 수도 있다.

In [59]:
tips.pivot_table(values="tip_pct", index="sex", columns="smoker", aggfunc="count", margins=True)


smoker,Yes,No,All
sex,,,
Male,60,97,157
Female,33,54,87
All,93,151,244


이제 성별과 흡연 여부에 따른 평균 팁 비율을 살펴본다.

In [60]:
tips[['sex', 'tip_pct']]

,sex,tip_pct
0,Female,0.059447
1,Male,0.160542
2,Male,0.166587
3,Male,0.139780
4,Female,0.146808
...,...,...
239,Male,0.203927
240,Female,0.073584
241,Male,0.088222
242,Male,0.098204


In [61]:
tips.groupby("sex")['tip_pct'].mean()

sex
Male      0.157651
Female    0.166491
Name: tip_pct, dtype: float64

In [62]:
tips.groupby("sex")[['tip_pct']].mean()

,tip_pct
sex,
Male,0.157651
Female,0.166491


In [63]:
tips.groupby("smoker")[["tip_pct"]].mean()

,tip_pct
smoker,
Yes,0.163196
No,0.159328


pivot_table 명령을 사용할 수도 있다.



In [64]:
tips.pivot_table(values="tip_pct", index="sex") # default 평균

,tip_pct
sex,
Male,0.157651
Female,0.166491


In [65]:
tips.pivot_table("tip_pct", ["sex", "smoker"]) #다중인덱스

tip_pct
sex    smoker          
Male   Yes     0.152771
       No      0.160669
Female Yes     0.182150
       No      0.156921

In [66]:
tips.pivot_table("tip_pct", "sex", "smoker") # data, index, columns

smoker,Yes,No
sex,,
Male,0.152771,0.160669
Female,0.182150,0.156921


여성 혹은 흡연자의 팁 비율이 높은 것을 볼 수 있다.
하지만 이 데이터에는 평균을 제외한 분산(variance) 등의 다른 통계값이 없으므로 describe 명령으로 여러가지 통계값을 한 번에 알아본다.

In [67]:
tips.groupby("sex")[["tip_pct"]].describe()

tip_pct                                                              \
         count      mean       std       min       25%       50%       75%   
sex                                                                          
Male     157.0  0.157651  0.064778  0.035638  0.121389  0.153492  0.186240   
Female    87.0  0.166491  0.053632  0.056433  0.140416  0.155581  0.194266   

                  
             max  
sex               
Male    0.710345  
Female  0.416667

In [68]:
tips.groupby("smoker")[["tip_pct"]].describe()

tip_pct                                                              \
         count      mean       std       min       25%       50%       75%   
smoker                                                                       
Yes       93.0  0.163196  0.085119  0.035638  0.106771  0.153846  0.195059   
No       151.0  0.159328  0.039910  0.056797  0.136906  0.155625  0.185014   

                  
             max  
smoker            
Yes     0.710345  
No      0.291990

In [69]:
tips.groupby(["sex", "smoker"])[["tip_pct"]].describe()

tip_pct                                                    \
                count      mean       std       min       25%       50%   
sex    smoker                                                             
Male   Yes       60.0  0.152771  0.090588  0.035638  0.101845  0.141015   
       No        97.0  0.160669  0.041849  0.071804  0.131810  0.157604   
Female Yes       33.0  0.182150  0.071595  0.056433  0.152439  0.173913   
       No        54.0  0.156921  0.036421  0.056797  0.139708  0.149691   

                                   
                    75%       max  
sex    smoker                      
Male   Yes     0.191697  0.710345  
       No      0.186220  0.291990  
Female Yes     0.198216  0.416667  
       No      0.181630  0.252672

연습 문제

1. 팁의 비율이 요일과 점심/저녁 여부, 인원수에 어떤 영향을 받는지 살펴본다.
2. 어떤 요인이 가장 크게 작용하는지 판단할 수 있는 방법이 있는가?

In [70]:
tips

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


In [76]:
# 1. 팁의 비율이 요일과 점심/저녁 여부, 인원수에 어떤 영향을 받는지 살펴본다
# day, time, size에 따른 평균 팁 계산

tip_pct_by_day_time_size = tips.pivot_table(values = 'tip', index = ['day', 'time', 'size'], columns = 'sex', aggfunc = 'mean')

print(tip_pct_by_day_time_size)

sex                   Male    Female
day  time   size                    
Thur Lunch  1          NaN  1.830000
            2     2.647083  2.204783
            3     3.090000  2.295000
            4     4.000000  4.363333
            5     5.000000       NaN
            6     6.700000  4.600000
     Dinner 2          NaN  3.000000
Fri  Lunch  1     1.920000       NaN
            2     1.890000  2.660000
            3          NaN  3.000000
     Dinner 2     2.750000  2.810000
            4     4.730000       NaN
Sat  Dinner 1          NaN  1.000000
            2     2.419412  2.693158
            3     3.740000  3.948000
            4     4.370000  2.770000
            5     3.000000       NaN
Sun  Dinner 2     2.816250  2.820000
            3     3.182222  3.028333
            4     4.001429  4.390000
            5     3.500000  5.140000
            6     5.000000       NaN


In [87]:
# 2. 어떤 요인이 가장 크게 작용하는지 판단할 수 있는 방법이 있는가?
# 가장 큰 팁을 가진 조합 찾기
print(tip_pct_by_day_time_size.idxmax()) # 데이터프레임에서 가장 큰 값을 가진 조합의 인덱스를 반환

sex
Male      (Thur, Lunch, 6)
Female    (Sun, Dinner, 5)
dtype: object


이번에는 각 그룹에서 가장 많은 팁과 가장 적은 팁의 차이를 알아보자.
이 계산을 해 줄 수 있는 그룹연산 함수가 없으므로 함수를 직접 만들고 agg 메서드를 사용한다.

In [80]:
def peak_to_peak(x):
    return x.max() - x.min()

tips.groupby(["sex", "smoker"])[["tip"]].agg(peak_to_peak)

tip
sex    smoker      
Male   Yes     9.00
       No      7.75
Female Yes     5.50
       No      4.20

만약 여러가지 그룹연산을 동시에 하고 싶다면 다음과 같이 리스트를 이용한다.



In [81]:
tips.groupby(["sex", "smoker"])[["total_bill"]].agg(["mean", peak_to_peak])

total_bill             
                    mean peak_to_peak
sex    smoker                        
Male   Yes     22.284500        43.56
       No      19.791237        40.82
Female Yes     17.977879        41.23
       No      18.105185        28.58

만약 데이터 열마다 다른 연산을 하고 싶다면 열 라벨과 연산 이름(또는 함수)를 딕셔너리로 넣는다.

In [83]:
tips

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


In [84]:
tips.groupby(["sex", "smoker"]).agg({'tip_pct': 'mean' , 'total_bill': peak_to_peak}) 

tip_pct  total_bill
sex    smoker                      
Male   Yes     0.152771       43.56
       No      0.160669       40.82
Female Yes     0.182150       41.23
       No      0.156921       28.58

* 피봇테이블

In [85]:
tips.pivot_table(values = ['tip_pct', 'size'], index=['sex', 'day'], columns='smoker')

size             tip_pct          
smoker            Yes        No       Yes        No
sex    day                                         
Male   Thur  2.300000  2.500000  0.164417  0.165706
       Fri   2.125000  2.000000  0.144730  0.138005
       Sat   2.629630  2.656250  0.139067  0.162132
       Sun   2.600000  2.883721  0.173964  0.158291
Female Thur  2.428571  2.480000  0.163073  0.155971
       Fri   2.000000  2.500000  0.209129  0.165296
       Sat   2.200000  2.307692  0.163817  0.147993
       Sun   2.500000  3.071429  0.237075  0.165710

In [86]:
tips.pivot_table('size', ['time', 'sex', 'smoker'], 'day',
                 aggfunc='sum', fill_value=0)

day                   Thur  Fri  Sat  Sun
time   sex    smoker                     
Lunch  Male   Yes       23    5    0    0
              No        50    0    0    0
       Female Yes       17    6    0    0
              No        60    3    0    0
Dinner Male   Yes        0   12   71   39
              No         0    4   85  124
       Female Yes        0    8   33   10
              No         2    2   30   43

연습 문제

타이타닉 승객 데이터를 이용하여 다음 분석을 실시하라. 데이터는 다음과 같이 받을 수 있다.

titanic = sns.load_dataset("titanic")


1. `qcut` 명령으로 세 개의 나이 그룹을 만든다.
2. 성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산한다. 
행에는 성별 및 나이 그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다.
생존률은 해당 그룹의 생존 인원수를 전체 인원수로 나눈 값이다.
3. 성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다.

In [91]:
titanic = sns.load_dataset("titanic")
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [125]:
# 1. `qcut` 명령으로 세 개의 나이 그룹을 만든다.
titanic["연령구간"] = pd.qcut(titanic.age, 3, labels= ["젊은", "중년", "노인"]).astype(str)
titanic["연령구간"]

0       젊은
1       노인
2       중년
3       노인
4       노인
      ... 
886     중년
887     젊은
888    nan
889     중년
890     중년
Name: 연령구간, Length: 891, dtype: object

In [ ]:
# 성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산한다. 
# 행에는 성별 및 나이 그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다.
# 생존률은 해당 그룹의 생존 인원수를 전체 인원수로 나눈 값이다.

In [128]:
# 생존율 함수 만들기
def pct_alive(x):
    return x.sum() / len(titanic)
# 그룹바이
titanic.groupby(["sex", "연령구간", "class"])[["survived"]].agg(pct_alive)

survived
sex    연령구간 class           
female nan  First   0.010101
            Second  0.002245
            Third   0.028058
       노인   First   0.048260
            Second  0.020202
            Third   0.004489
       젊은   First   0.023569
            Second  0.022447
            Third   0.033670
       중년   First   0.020202
            Second  0.033670
            Third   0.014590
male   nan  First   0.005612
            Second  0.002245
            Third   0.010101
       노인   First   0.026936
            Second  0.002245
            Third   0.003367
       젊은   First   0.005612
            Second  0.011223
            Third   0.019080
       중년   First   0.012346
            Second  0.003367
            Third   0.020202

In [131]:
titanic.groupby(["sex", "연령구간", "class"])[["survived"]].agg(pct_alive).unstack("class") #class를 반시계방향으로 돌려야하니까 unstack

survived                    
class           First    Second     Third
sex    연령구간                              
female nan   0.010101  0.002245  0.028058
       노인    0.048260  0.020202  0.004489
       젊은    0.023569  0.022447  0.033670
       중년    0.020202  0.033670  0.014590
male   nan   0.005612  0.002245  0.010101
       노인    0.026936  0.002245  0.003367
       젊은    0.005612  0.011223  0.019080
       중년    0.012346  0.003367  0.020202

In [130]:
# 피봇테이블 만들기
titanic.pivot_table(values = 'survived', index = ['sex', '연령구간'], columns = 'class', aggfunc=pct_alive)

class           First    Second     Third
sex    연령구간                              
female nan   0.010101  0.002245  0.028058
       노인    0.048260  0.020202  0.004489
       젊은    0.023569  0.022447  0.033670
       중년    0.020202  0.033670  0.014590
male   nan   0.005612  0.002245  0.010101
       노인    0.026936  0.002245  0.003367
       젊은    0.005612  0.011223  0.019080
       중년    0.012346  0.003367  0.020202